In [1]:
import os
import sys
import types
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sas

import read_clean_quote  as rcq

In [2]:
rolling_list = [3,5,10,15,20,25,30]

In [4]:
tempDf = rcq.read_quote_data('IBM')

rcq.fill_quote_gaps( tempDf )
rcq.fix_splits( tempDf )

diffs = rcq.generate_differentials( tempDf               ) # Differentials between c,ac, o, h, l, v
tempDf = tempDf.join( diffs, how='inner' )
rolls = rcq.generate_rolling_close( tempDf, rolling_list ) # Rolling averages of adj c, listed as C_mean_n and C_std_10
moms  = rcq.generate_momentum_close(tempDf, rolling_list ) # Momentum between day and days offset in list, Momentum_n
rsis  = rcq.generate_rsi(           tempDf, rolling_list ) # RSIs over n days, RSI_n

# Join everything, or can leave as seperate 
tempDf = tempDf.join( rolls, how='inner' )
tempDf = tempDf.join(  moms, how='inner' )
tempDf = tempDf.join(  rsis, how='inner' )
tempDf = tempDf.drop( ['Open','High','Low','Volume','Close'], axis=1 )

In [5]:
tempDf.head()

,Adj Close,Diff_co,Diff_hl,Diff_v,Close_mean_3,Close_std_3,Close_mean_5,Close_std_5,Close_mean_10,Close_std_10,...,Momentum_20,Momentum_25,Momentum_30,RSI_3,RSI_5,RSI_10,RSI_15,RSI_20,RSI_25,RSI_30
Date,,,,,,,,,,,,,,,,,,,,,
2017-03-01,181.949997,0.008145,0.013963,-0.095493,180.389999,1.367222,180.834000,1.146965,180.935997,0.861860,...,0.050817,0.042573,0.092314,50.206088,50.071219,50.070191,50.030241,49.972440,50.100510,50.218339
2017-02-28,179.820007,0.002453,0.007136,-0.108796,180.190002,1.026308,180.673999,0.998563,180.753998,0.814822,...,0.030955,0.059708,0.083075,50.164592,50.034368,50.019917,49.976369,50.017672,50.044073,50.108271
2017-02-27,179.399994,-0.009879,0.010873,0.123692,180.799998,1.221682,180.761997,0.920855,180.707997,0.883499,...,0.019845,0.060208,0.076621,50.113991,50.112437,50.007195,49.915102,50.045903,49.961848,50.182521
2017-02-24,181.350006,0.006326,0.008879,0.452033,181.383331,0.251660,181.015997,0.552884,180.635997,1.020633,...,0.023082,0.095761,0.089621,49.944889,50.053852,49.915490,49.851535,50.078652,49.916914,50.157921
2017-02-23,181.649994,-0.001923,0.008681,-0.253194,181.019994,0.704059,181.031995,0.565967,180.221997,1.448711,...,0.026902,0.097640,0.106128,50.026926,50.020364,49.819682,49.756789,50.044392,50.037754,50.050590


In [74]:
# Data should be cleaned, need to plot some trends and check for correlation
# [ 'Diff_co', 'Diff_ao', 'Diff_hl', 'Diff_v'] ]
# [ 'Close_mean_n', 'Close_std_n' ]
# [ 'Momentum_n' ]
#
# Diff co, v, have different info, co same as hl basically
# Momentum 3 correlated with co, 5 a little less, 10 at limit
# HUGE autocorrelation with momentum, long momentum really tracks trends, until at/past scale of momentum

In [64]:
test_1 = 'Momentum_5'
test_2 = 'Momentum_10'

In [65]:
tempDf[0:1000].plot.scatter( x=test_1, y=test_2, color='k' )
plt.show()

In [62]:
corrlist = ['Diff_co','Momentum_3','Momentum_5','Momentum_10','Momentum_15','Momentum_20','Momentum_25','Momentum_30']

In [73]:
for col in corrlist:
    print tempDf[col].autocorr(30)

0.0229732268304
-0.02241122096
-0.0200966782939
0.0314853719823
0.0489461803723
0.0562224030535
0.0549889204033
0.0457616299563


In [5]:
from sklearn.decomposition import PCA

In [38]:
foo = moms.copy()
#foo = foo.join( rolls[test_lab] )

In [39]:
myPCA = PCA( n_components = 5 )
myPCA.fit( foo )
print myPCA.explained_variance_ratio_

[ 0.77970679  0.1020569   0.04504036  0.02603589  0.02113785]


In [12]:
myPCA = PCA( n_components = 4 )
myPCA.fit( foo )
print myPCA.explained_variance_ratio_

[  8.17106106e-01   1.82661440e-01   1.35190310e-04   9.72636776e-05]
